# OR-Tools를 이용한 순회외판원(여행사 영업 담당자 문제, Traveling Salesman Problem) 문제 해결

아래 링크에서 더 자세한 설명을 확인할 수 있습니다. (Google OR-Tools 제공)

https://developers.google.com/optimization/routing/tsp?hl=ko

In [1]:
# 파이썬에서 제공하는 패키지 매니저 `pip`를 이용해 Google OR-Tools 설치합니다 (Python)
# 코드 앞의 느낌표(!)가 붙으면 파이썬 코드가 아니라 터미널(또는 콘솔, 명령 프롬프트 등)에서 명령어를 실행하는 것과 같습니다
!pip install --upgrade ortools

In [2]:
# 필요한 OR-Tools 패키지 불러오기
# 위 셀에서 설치가 되었는데도 `ModuleNotFoundError`가 발생하는 경우 [런타임] -> [런타임 다시 시작] 후 재시도 해볼것
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [3]:
# 거리 행렬: 0행1열 = 0번째 도시와 1번째 도시 사이 거리 (2451), 1행2열 = 1번째도시와 2번째 도시 사이 거리 (1745)
def create_data_model():
    """문제 해결을 위한 데이터를 거리 행렬 형태로 저장합니다."""
    data = {}
    data["distance_matrix"] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]
    data["num_vehicles"] = 1  # 차량(외판원)이 몇개(몇명)인지? (1명)
    data["depot"] = 0  # 시작할 도시 번호 (0번째 도시)
    return data



In [4]:
data = create_data_model()  # 데이터를 생성합니다
manager = pywrapcp.RoutingIndexManager(  # OR-Tools에서 제공하는 경로 탐색 매니저(Routing Index Manager)를 선언합니다.
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]  # 각각 거리, 차량 개수, 도시 번호
)
routing = pywrapcp.RoutingModel(manager)  # 경로 탐색 모델(Routing Model)을 선언합니다


def distance_callback(from_index, to_index):
    """두 도시 사이의 거리를 반환하는 함수."""
    # 경로 탐색 매니저에 저장된 도시 번호로부터 거리값을 가져옵니다.
    from_node = manager.IndexToNode(from_index)  # 출발할 도시
    to_node = manager.IndexToNode(to_index)  # 도착할 도시
    return data["distance_matrix"][from_node][to_node]

# 거리를 반환하는 함수를 경로 탐색 모델(Routing Model)에 등록합니다
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [5]:
import time  # 시간 측정을 위한 파이썬 기본 패키지

# 경로 탐색을 위해 필요한 값들을 설정해줍니다.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()  # 경로 탐색을 위한 기본 파라미터 변수들
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC  # 경로 탐색 전략을 선택 (가장 가까운 도시로 이동하는 전략)
)

def print_solution(manager, routing, solution):
    """콘솔에 탐색한 경로를 표시하는 함수"""
    print(f"경로 길이: {solution.ObjectiveValue()} 마일")
    index = routing.Start(0)
    plan_output = ""
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        print(plan_output)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        time.sleep(1)
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}miles\n"

solution = routing.SolveWithParameters(search_parameters)  # 경로 탐색!
if solution:
    print_solution(manager, routing, solution)  # 콘솔에 출력

경로 길이: 7293 마일
 0 ->
 0 -> 7 ->
 0 -> 7 -> 2 ->
 0 -> 7 -> 2 -> 3 ->
 0 -> 7 -> 2 -> 3 -> 4 ->
 0 -> 7 -> 2 -> 3 -> 4 -> 12 ->
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 ->
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 ->
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 ->
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 ->
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 10 ->
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 10 -> 5 ->
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 10 -> 5 -> 9 ->
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 10 -> 5 -> 9 -> 0

